# COVID-19 Overview
> Tracking coronavirus total cases, deaths and new cases by country. Additionally, a detailed view is provided for the US(by state), Europe

- comments: true
- author: Pratap Vardhan
- categories: [overview, interactive]
- image: images/covid-overview.png
- permalink: /covid-overview/
- hide: false

In [1]:
#hide
print('''
Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/
''')


Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/



In [2]:
#hide
import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [3]:
#hide

# FETCH
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + 'overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [4]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('deaths')
dft_recovered = get_frame('recovered')
LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]


dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_5ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_5ago].sum()

In [5]:
#hide
df_table = (pd.DataFrame(dict(Cases=dfc_cases, Deaths=dfc_deaths, PCases=dfp_cases, PDeaths=dfp_deaths))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
df_table.rename(columns={'index': 'Country/Region'}, inplace=True)
for c in 'Cases, Deaths'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table.head(15)

Country/Region     Cases  Deaths    PCases  PDeaths  Cases (+)  \
0               US  34281864  610177  34067912   608884     213952   
1            India  31293062  419470  31106065   413609     186997   
2           Brazil  19523711  547016  19342448   541266     181263   
3           France   5996060  111749   5917397   111657      78663   
4           Russia   5979027  149012   5860113   145222     118914   
5   United Kingdom   5626745  129266   5407428   128960     219317   
6           Turkey   5563903   50761   5522039    50488      41864   
7        Argentina   4812351  103074   4749443   101434      62908   
8         Colombia   4692570  117836   4621260   115831      71310   
9            Italy   4302393  127920   4284332   127864      18061   
10           Spain   4249258   81194   4100222    81096     149036   
11         Germany   3758425   91505   3751253    91369       7172   
12            Iran   3623840   88063   3501079    86966     122761   
13       Indonesia   3033339   79032   2832755    72489     200584   
14          Poland   2881840   75231   2881355    75212        485   

    Deaths (+)  Fatality Rate      Continent  
0         1293            1.8  North America  
1         5861            1.3           Asia  
2         5750            2.8  South America  
3           92            1.9         Europe  
4         3790            2.5         Europe  
5          306            2.3         Europe  
6          273            0.9           Asia  
7         1640            2.1  South America  
8         2005            2.5  South America  
9           56            3.0         Europe  
10          98            1.9         Europe  
11         136            2.4         Europe  
12        1097            2.4           Asia  
13        6543            2.6           Asia  
14          19            2.6         Europe

In [6]:
#hide
#delete problematic countries from table
df_table = df_table[~df_table['Country/Region'].isin(['Cape Verde', 'Cruise Ship', 'Kosovo'])]

In [7]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Cases (+)', 'Deaths (+)']
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_5ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu}
summary

{'updated': Timestamp('2021-07-22 00:00:00'),
 'since': Timestamp('2021-07-17 00:00:00'),
 'Cases': 192466040,
 'Deaths': 4133535,
 'Cases (+)': 2575462,
 'Deaths (+)': 52177,
 'China Cases': 104489,
 'China Deaths': 4848,
 'China Cases (+)': 232,
 'China Deaths (+)': 0,
 'US Cases': 34281864,
 'US Deaths': 610177,
 'US Cases (+)': 213952,
 'US Deaths (+)': 1293,
 'EU Cases': 50394919,
 'EU Deaths': 1122319,
 'EU Cases (+)': 706810,
 'EU Deaths (+)': 4916}

In [8]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  1/27/20  1/28/20  \
Country/Region                                                                  
Afghanistan           0        0        0        0        0        0        0   
Albania               0        0        0        0        0        0        0   
Algeria               0        0        0        0        0        0        0   
Andorra               0        0        0        0        0        0        0   
Angola                0        0        0        0        0        0        0   

                1/29/20  1/30/20  1/31/20  ...  7/13/21  7/14/21  7/15/21  \
Country/Region                             ...                              
Afghanistan           0        0        0  ...        0     1990     1210   
Albania               0        0        0  ...       11        8       13   
Algeria               0        0        0  ...      941      914     1109   
Andorra               0        0        0  ...       12        0       72   
Angola                0        0        0  ...       83      189      203   

                7/16/21  7/17/21  7/18/21  7/19/21  7/20/21  7/21/21  7/22/21  
Country/Region                                                                 
Afghanistan           0        0        0     3636      925        0      769  
Albania              18       18       21       11       43       23       34  
Algeria            1197     1107     1099     1177     1298     1221     1208  
Andorra              34        0        0       86       20        0       85  
Angola              101       76       98      101      155      166      178  

[5 rows x 548 columns]

In [9]:
#hide_input
template = Template(get_template(paths['overview']))
html = template.render(
    D=summary, table=df_table,  # REMOVE .head(20) to see all values
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    np=np, pd=pd, enumerate=enumerate)
HTML(f'<div>{html}</div>')

Visualizations by [Pratap Vardhan](https://twitter.com/PratapVardhan)[^1]

[^1]: Source: ["COVID-19 Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). Link to [notebook](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-overview.ipynb), [orignal interactive](https://gramener.com/enumter/covid19/)